# EddyPro API

This notebook gives an outline for working with the API.

* First step is to define your input primary parameters.  You need to give a siteID, everything else is *optional*.

    * kwargs (dict of all optional settings):
        * dateRange (if not provided, will run all files in the current year)
        * fileType (defaults to .ghg) must specify otherwise for 
        * metadataTemplate (only required for .dat files)



In [29]:
# Example run for BB on one day in 2024
siteID = 'BB'
kwargs = {'dateRange':['2024-04-01','2024-04-02']}

# # # Example run for All BBS data
# siteID = 'BBS'
# kwargs = {
#     'dateRange':['2023-06-01','2024-05-31'],
#     'fileType':'dat',
#     'metaDataTemplate':'C:/highfreq/BBS/TOA5_BBS.FLUX_2023_06_14_1500.metadata'
# }

## Create Optional Auxillary Input Data

* It's best practice to give EddyPro Biomet data (temperature, radiation, etc.) and any dynamic metadata (canopy height) in a .csv file.  Biomet.Net users can create the files from database traces using the code below.  You can also create these files yourself

    * Biomet files: https://www.licor.com/env/support/EddyPro/topics/biomet-data-format.html
    * Dynamic metadata files: https://www.licor.com/env/support/EddyPro/topics/dynamic-metadata.html

In [30]:
# Create biomet and dynamicMetadata.csv files
import os
import sys
import importlib

BiometNet = 'C:/Biomet.net/Python/'

# UBC Micromet users can 
if sys.path[0]!=BiometNet:
    sys.path.insert(0,BiometNet)

wd = [p for p in sys.path if p != BiometNet][0]

import csvFromBinary as cfb
importlib.reload(cfb)

if 'dateRange' in kwargs:
    dateRange = kwargs['dateRange']
else:
    dateRange = None

tasks=os.path.abspath(wd+'/config_files/EP_auxillary_data_defs.yml')
auxilaryDpaths=cfb.makeCSV(siteID,dateRange,tasks,
                           outputPath=f'C:/highfreq/{siteID}/auxilaryData',stage='Second')
for key,value in auxilaryDpaths.items():
    kwargs[key] = value

Initializing tasks for BB over: ['2024-04-01', '2024-04-02']
PPFD_IN_1_1_1 missing, outputting NaNs
See output: C:/highfreq/BB/auxilaryData/BB_biometData_202404010000_202404020000.csv
canopy_height missing, outputting NaNs
See output: C:/highfreq/BB/auxilaryData/BB_dynamicMetadata_202404010000_202404020000.csv
All tasks completed successfully


In [42]:
import eddyProAPI
importlib.reload(eddyProAPI)
pre = eddyProAPI.preProcessing(siteID,**kwargs)

Searching C:/highfreq//BB/raw/2024\04
[████████████████████████████████████████████████████████████] 144/144

Files Search Complete, time elapsed:  0.8421986103057861
Reading Data
Reading Complete, time elapsed:  0.026254892349243164


In [43]:

import eddyProAPI
import importlib
importlib.reload(eddyProAPI)


# Cospectral correction options
hf_meth = {
    'Moncrieff et al. (1997)':1,
    'Horst (1997)':2,
    'Ibrom et al. (2007)':3,
    'Fratini et al. (2012)':4, # Recommended for closed path systems, when sufficient data (>1 month) are available
    'Massman (2000, 2001)':5
}

if siteID == 'BBS':
    userDefinedEddyProSettings = {
        'Project':{
            'hf_meth':'1'
            },
        'RawProcess_Settings':{
            'v_offset':0.08,
            'u_offset':-0.04
            },
        }
    kwargs['userDefinedEddyProSettings'] = userDefinedEddyProSettings

eP = eddyProAPI.runEP(siteID,**kwargs)

Creating c:\MM_Py\EddyPro_API\temp\group_1_rp_A.eddypro for 13 files
28
Creating c:\MM_Py\EddyPro_API\temp\group_1_rp_B.eddypro for 12 files
28
Creating c:\MM_Py\EddyPro_API\temp\group_1_rp_C.eddypro for 12 files
28
Creating c:\MM_Py\EddyPro_API\temp\group_1_rp_D.eddypro for 12 files
28
Creating c:\MM_Py\EddyPro_API\temp\group_1_fcc.eddypro for 49 files
28
Initiating EddyPro Runs for group 1 on 4 cores at normal priority
['c:\\MM_Py\\EddyPro_API\\temp\\group_1_rp_A.eddypro', 'c:\\MM_Py\\EddyPro_API\\temp\\group_1_rp_B.eddypro', 'c:\\MM_Py\\EddyPro_API\\temp\\group_1_rp_C.eddypro', 'c:\\MM_Py\\EddyPro_API\\temp\\group_1_rp_D.eddypro']
[████████████████████████████████████████████████████████████] 4/4

Remember to update project ID>//////>??????
